In [1]:
import sys
import os
from IPython.display import Audio, display

sys.path.append("..")

In [ ]:
Audio("../datasets/vcc2016_training/SF1/100001.wav")

In [ ]:
import librosa
import numpy as np

def read_audio_spectum(filename, n_fft=24):
    x, fs = librosa.load(filename)
    S = librosa.stft(x, n_fft)
    p = np.angle(S)
    S = np.log1p(np.abs(S[:, :430]))
    return S, fs

def load_data(path):
    data = dict()
    
    for sub_dir in os.listdir(path):
        data[sub_dir] = list()
        
        for file in os.listdir(path + sub_dir):
            x, fs = read_audio_spectum(path + sub_dir + "/" + file)
            data[sub_dir].append(x)
            
    return data

In [ ]:
data = load_data("../datasets/vcc2016_training/")

In [2]:
from lib.zoo import Sound2SoundNet, Discriminator1D

In [6]:
%%time

genA2B = Sound2SoundNet([None, 23, 100])
genB2A = Sound2SoundNet([None, 23, 100])

realA = genA2B.input_var
realB = genB2A.input_var

discA = Discriminator1D(genA2B, real_input_var=realA)
discB = Discriminator1D(genB2A, real_input_var=realB)

CPU times: user 8.99 s, sys: 294 ms, total: 9.28 s
Wall time: 9.32 s


In [8]:
cost = T.abs_(realA - genB2A.output_var).mean()

In [ ]:
updates = lasagne.updates.adam(cost, genB2A.params)
learn = theano.function([genB2A.input_var], cost, updates=updates)